In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

In [ ]:
base <- "193.175.238.88:8080/api/organizations/"
call1 <- paste(base,"all/", sep="")
get_organizations_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
# In the table 'get_organizations_df' you can see all the extracted organizations with respected twitter, facebook and wikipedia ids.

tables_rjs(get_organizations_df[get_organizations_df$category == 'political',], jupyter = TRUE)

In [ ]:
# Please choose a subcategory from the table above and input the `party` and `aggregation` variables below:
party <- "spd"
aggregation <- "month" # aggregation can be by day ('day'), week ('week') or month('month')

base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"
party_df <- get_organizations_df[get_organizations_df$subcategory == party & !is.null(get_organizations_df$wp_ids) ,]
chobs <- data.frame()
wikiwho <- data.frame()
for (wp_id in party_df$wp_ids) {
    for (wp in wp_id) {
        call_chobs <- paste(base,"wikipedia/chobs/organizations/page_id/", toString(wp), "/?aggregate_by=", aggregation, sep="")
        wikiwho <- rbind(wikiwho, as.data.frame(fromJSON(content(GET(paste(base_ww, wp, "/", sep="")), "text", encoding="UTF-8"), flatten = TRUE)))
        if (length(fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
            chobs_unique <- as.data.frame(fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"), flatten = TRUE))
            chobs_unique$wp_user_id = wp
            chobs <- rbind(chobs, chobs_unique)
            }
    }
}
wikiwho$undos <- wikiwho$editions.dels + wikiwho$editions.reins
wikiwho <- summarise(group_by(wikiwho, editions.year_month), undos = sum(undos),revisions = sum(editions.revisions), conflicts = sum(editions.conflicts))
chobs <- summarise(group_by(chobs, labels), values = sum(values), response_type = aggregation)
merged = merge(chobs, wikiwho, by.x='labels', by.y='editions.year_month')

In [ ]:
#plotting
options(repr.plot.width=6, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = values, color="Change objects", group=response_type)) +
  geom_line(aes(y = revisions, color="Revisions", group=response_type)) +
  labs(title = "Wikipedia (chobs and revisions)", x = "Date", y = "N") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 year")

In [ ]:
#plotting
options(repr.plot.width=6, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = undos, color="Undos", group=response_type)) +
  geom_line(aes(y = conflicts, color="Conflicts", group=response_type)) +
  labs(title = "Wikipedia (undos and conflicts)", x = "Date", y = "N") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 year")